## ONLY if running on Colaboratory, run this cell first (once)

In [ ]:
!git clone https://github.com/pie3636/newsjam.git
!mv newsjam/* .

## Install missing modules if needed (only run once)

In [2]:
!python -m pip install -r requirements.txt
!python -m spacy download fr_core_news_sm
# Note: You'll have to restart the kernel/runtime after running this cell

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch
C:\Users\PC HP 2020\AppData\Local\Programs\Python\Python310\python.exe: No module named spacy


## Imports (only run once)

In [1]:
# MLSUM Corpus
from datasets import load_dataset

# Loading article data
import json

# Our packages
from eval.rouge_l import RougeLEval
from eval.bert_eval import BERT_Eval
from eval.time import TimeEval

from summ.lsa import LSASummarizer
from summ.bert_embed import BertEmbeddingsSummarizer

from tqdm import tqdm

dataset = load_dataset('mlsum', 'fr')

rouge_l = RougeLEval()
bert = BERT_Eval()
timer = TimeEval()
lsa_summ = LSASummarizer()
flaubert_summ = BertEmbeddingsSummarizer('flaubert/flaubert_large_cased')
camembert_summ = BertEmbeddingsSummarizer('camembert/camembert-large')

Reusing dataset mlsum (/Users/josephkeenan/.cache/huggingface/datasets/mlsum/fr/1.0.0/77f23eb185781f439927ac2569ab1da1083195d8b2dab2b2f6bbe52feb600688)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at flaubert/flaubert_large_cased were not used when initializing FlaubertModel: ['pred_layer.proj.bias', 'p

## Summarize a single article

In [3]:
# Pick an article and its reference summary
article = dataset['test']['text'][54]
ref_summ = dataset['test']['summary'][54]

# Computes the summary and evaluation
# timer.evaluate_one(article, BertEmbeddingsSummarizer, 'camembert/camembert-large')

## Summarize a series of articles

In [73]:
texts = dataset['test']['text'][:50]
ref_summs = dataset['test']['summary'][:50]

# Here we pick 5 articles
# gen_summs = []
# for text in tqdm(texts[:5]):
#     gen_summs.append(flaubert_summ.get_summary(text))

# scores1, scores2 = rouge_l.evaluate_many(ref_summs, gen_summs, 5)
# results = rouge_l.get_results(scores1, scores2)

# for k, v in results.items():
#     print(k.ljust(25), round(v*100, 3), '%')

timer.evaluate_many(texts, LSASummarizer)

100%|███████████████████████████████████████████| 50/50 [01:33<00:00,  1.86s/it]


1.8826856641600034

#### Optional: Save generated summaries to file

In [ ]:
with open('generated.txt', 'w') as f:
    for summ1, summ2 in tqdm(gen_summs):
        f.write(summ1)
        f.write('\n\n')
        f.write(summ2)
        f.write('\n\n')

## Summarize a series of scraped articles

In [31]:
with open('data/actu_preliminary.json', 'r', encoding='utf-8') as jsonfile:
    data = json.load(jsonfile)

texts = [article['text'] for article in data]
ref_summs = [article['summary'] for article in data]

gen_summs = []
for text in tqdm(texts):
    gen_summs.append(lsa_summ.get_summary(text))

scores1, scores2 = rouge_l.evaluate_many(ref_summs, gen_summs)
results = rouge_l.get_results(scores1, scores2)

for k, v in results.items():
    print(k.ljust(25), round(v*100, 3), '%')

100%|███████████████████████████████████████████| 47/47 [00:00<00:00, 84.52it/s]

Long precision avg        50.43 %
Long recall avg           58.793 %
Long F1-score avg         53.661 %
Keyword precision avg     49.32 %
Keyword recall avg        58.105 %
Keyword F1-score avg      52.674 %


In [80]:
for x in gen_summs:
    print(x)

('Suivre L’académie de Toulouse a publié ce vendredi 15 octobre 2021 les chiffres du Covid-19 dans les établissements de la région.  \nActu Toulouse', 'académie toulouse publier vendredi 15 octobre 2021 chiffre Covid-19 établissement région  \nactualité toulouse')
('À lire aussi Saint-Étienne.\nPour ces raisons, la préfecture de la Loire a pris la décision d’alléger les mesures concernant le port du masque dans le département.', 'lire saint-étienne\nraison préfecture Loire prendre décision alléger mesure port masque département')
('Fin de la gratuité systématique des tests\n14 clusters en Pays de la Loire Le Maine-et-Loire voit son taux d’incidence repasser en dessous de la barre des 50 cas pour 100 000 habitants.', 'fin gratuité systématique test\n14 clusters Pays Loire maine-et-loire voir taux incidence repasser barre 50 cas 100 000 habitant')
('"Injustifié", "clivage irréparable" : le pass sanitaire étendu ne passe pas L\'élargissement du pass sanitaire dans tous les sites de loisir

Implementation of BERTScore

In [74]:
long_summs, short_summs, ref_summs, key_ref_sums =  bert.split_summs(texts, ref_summs)

In [77]:
for x in long_summs:
    print(x, x.index)

L <built-in method index of str object at 0x7fb5a018b8b0>
M <built-in method index of str object at 0x7fb5a01fa670>
A <built-in method index of str object at 0x7fb5a01fa730>
L <built-in method index of str object at 0x7fb5a018b8b0>
L <built-in method index of str object at 0x7fb5a018b8b0>
R <built-in method index of str object at 0x7fb5a01e4ef0>
D <built-in method index of str object at 0x7fb5a01afcb0>
L <built-in method index of str object at 0x7fb5a018b8b0>
T <built-in method index of str object at 0x7fb5a01b6830>
L <built-in method index of str object at 0x7fb5a018b8b0>
L <built-in method index of str object at 0x7fb5a018b8b0>
U <built-in method index of str object at 0x7fb5a02fb1f0>
Q <built-in method index of str object at 0x7fb5a0330cf0>
S <built-in method index of str object at 0x7fb5a01b67f0>
J <built-in method index of str object at 0x7fb5a0330cb0>
« <built-in method index of str object at 0x7fb5a06c5620>
L <built-in method index of str object at 0x7fb5a018b8b0>
D <built-in me

In [78]:
x = 15
long_summs[10] = 'B'
print(long_summs[x])
print(len(long_summs[x]))
print()
print(ref_summs[x])
ref_summs[10] = '汉字'
print(len(ref_summs[x]))

«
1

Le neurobiologiste fait une lecture de la pièce de Molière (1672) située quelque part entre la psychanalyse freudienne et l’existentialisme pascalien. Subtil et drôle.
167


In [79]:
bert.bert_score(long_summs, short_summs, ref_summs, key_ref_sums, 10)

calculating scores...
computing bert embedding.



computing greedy matching.



done in 0.13 seconds, 7.93 sentences/sec
calculating scores...
computing bert embedding.



computing greedy matching.



done in 0.18 seconds, 5.55 sentences/sec


{'Long precision avg': '0.3093',
 'Long recall avg': '0.2388',
 'Long F1-score avg': '0.2743',
 'Keyword precision avg': '0.0504',
 'Keyword recall avg': '-0.3159',
 'Keyword F1-score avg': '-0.1531'}

In [ ]:
bert.get_matrix(long_summs, ref_summs, 4)